In [4]:
import re
import time
import pandas as pd
import selenium
from bs4 import BeautifulSoup
from dateutil import parser
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import requests

In [5]:
def scrapeMatch(url,date,browser):
    browser.get(url)
    html = browser.page_source
    bs = BeautifulSoup(html, 'html.parser')
    tables = bs.find_all('table')
    #for table in tables:
    final_columns = ['DATE','DATE_2','TEAM_HOME', 'TEAM_VISITOR', 'POINTS_HOME', 'POINTS_VISITOR']
    df_list = []
    tableList = []
    columns = ['DATE']
    for th in tables[0].find('thead').findAll('th'):
        text = th.text
        if text == '\xa0':
            text = 'TEAM'
        columns.append(text)
    for tr in tables[0].find('tbody').find_all('tr'):
        row = [date]
        for child in tr.findChildren(recursive=False):
            row.append(child.text)
        tableList.append(row)
    df = pd.DataFrame(columns=columns, data=tableList)
    df.drop(df.columns.difference(['DATE','TEAM','FINAL']), 1, inplace=True)
    df2 = df.unstack().to_frame().T
    df2.columns = final_columns
    df3=df2.loc[:, df2.columns.drop(['DATE_2'])]
    return df3

In [6]:
def check_winner(el):
    if el['POINTS_VISITOR'] != '' and el['POINTS_HOME'] != '':
        if el['POINTS_VISITOR'] > el['POINTS_HOME']:
            return el['TEAM_VISITOR']
        else:
            return el['TEAM_HOME']
    else:
        return None

In [7]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [9]:
chrome_options = Options()  
chrome_options.add_argument("--headless")
chrome_options.add_argument('log-level=3')
start_date = datetime.date(2022, 3, 1)
end_date = datetime.date(2022, 3, 28)
delta = datetime.timedelta(days=1)
df_list = []
with Chrome(ChromeDriverManager().install(),options=chrome_options) as browser:
    while (start_date <= end_date):
        date = start_date
        start_date += delta
        url = 'https://www.nba.com/games?date='+ date.strftime("%Y-%m-%d")
        response = requests.get(url)
        if response.status_code != 200:
            pass
        browser.get(url)
        html = browser.page_source
        bs = BeautifulSoup(html, 'html.parser')
        links = bs.find_all('a', {'href': re.compile(r'/game/[a-z-]+vs[a-z-0-9]+$')})
        refs = [link['href'] for link in links]
        refs = set(refs)
        for ref in refs:
            url = 'https://www.nba.com' + ref
            df_list.append(scrapeMatch(url,date,browser))
    df_final = pd.concat(df_list)
    df_final['WINNER'] = df_final.apply (lambda row: check_winner(row), axis=1)
    display(df_final)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
There is no [win32] chromedriver for browser 100.0.4896 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Francis Fernández\.wdm\drivers\chromedriver\win32\100.0.4896.60]
C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_8404/3034120236.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(ChromeDriverManager().install(),options=chrome_options) as browser:
C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_8404/1642095102.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['DATE','TEAM','FINAL']), 1, inplace=True)


,DATE,TEAM_HOME,TEAM_VISITOR,POINTS_HOME,POINTS_VISITOR,WINNER
0,2022-03-01,DET,WAS,113,116,WAS
0,2022-03-01,LAC,HOU,113,100,LAC
0,2022-03-01,GSW,MIN,114,129,MIN
0,2022-03-01,BKN,TOR,108,109,TOR
0,2022-03-01,DAL,LAL,109,104,DAL
...,...,...,...,...,...,...
0,2022-03-28,SAC,MIA,100,123,MIA
0,2022-03-28,ATL,IND,132,123,ATL
0,2022-03-28,GSW,MEM,95,123,GSW
0,2022-03-28,DEN,CHA,113,109,DEN


In [10]:
df_final.to_csv('dataset_pra1.csv')